<center><img src='img/ms_logo.jpeg' height=40% width=40%></center>

<center><h1>Outlier Detection, Sample Size, and Confidence Intervals</h1></center>

When you're designing an experiment, numbers matter.  After all, we want out experiments to be statistically valid--otherwise, we're just guessing.  In this notebook, we'll learn a method for detecting outliers in our data set called "Tukey Fences", named after famed statistician John Tukey.  

Next, we'll learn about confidence inteverals, sample size, and the relationship between the two.  We'll learn how to calculate confidence intervals based on sample size, as well as how to determine the minimum sample size needed in order to reach a specific confidence interval.  

Let's get started!

<center><h2>Outlier Detection</h2></center>

Recall that before we begin an experiment, we usually start by "cleaning" our dataset.  This step usually includes things like:

* Exploring our dataset(s) to get a feel for what changes need to be made to make it more usable
* Examining and standardizing the values within cells (converting "yes"/"no" answers to 1's and 0's, for example)
* Dealing with cells that contain NaNs (Null values)
* Organizing and structuring datasets as needed (for instance, combining many small datasets into one big one)
* Normalizing continuous data into z-scores with a mean of 0 and unit variance.  

Another major step we need to do at this point in the project is to detect **outliers**, and determine how to deal wit them.  Outliers are extreme values that can skew our dataset, sometimes giving us an incorrect picture of how things actually are in our dataset.  The hardest part of this is determining which data points are acceptable, and which ones constitute "outlier" status.  This is where "Tukey Fences" come into play!

### 1.5 x IQR

In order to find outliers, we first need a working definition of what constitutes an outlier.  Tukey suggested we calculate the range between the first quartile (25%) and  third quartile (75%) in the data, called the **interquartile range**.  We then multiply this value by 1.5.  To get the Fence for high values, add this value to the Q3 value.  Anything greater than this "Fence" value is considered an outlier.  Similarly, to get the Fence for low values, subtract 1.5 x IQR from Q1.  Anything less than this "Fence" value is also considered an outlier.  

Let's try an example!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(1547)
% matplotlib inline

In [2]:
# Generate a random normal distribution of 1000 samples with mean 100 and std_dev of 8
normal_dist = np.random.normal(100, 8, (1000)).astype('float64')
# Generate a random uniform distribution between 1 and 200 with 100 samples
uniform_dist = np.random.uniform(1, 200, (100)).astype('float64')
# Combine both distributions and store in and Pandas Series object
sample_dset = pd.Series(np.append(normal_dist, uniform_dist))
sample_dset.describe()

count    1100.000000
mean      101.190110
std        17.605682
min        10.235233
25%        94.658860
50%       100.340004
75%       106.459821
max       195.226466
dtype: float64

Now that we've created an ugly data set, let's see if we can identify some outliers.  

Start by calculating the **Inter-Quartile Range**: Q3 - Q1.

Next, calculate how far your fences are from the quartiles: f = IQR x 1.5

Finally, place your fences and filter for values outside them:  Lower Fence = Q1 - f, Upper Fence = Q3 + f

See if you can write write some code to filter for outliers in the `sample_dset` array we've just created.

In [6]:
# Get Locations for Q1 and Q3
q1 = 94.658860
q3 = 100.340004
f = (q3 - q1) * 1.5

# calculate fence locations
lower_fence = q1 - f
upper_fence = q3 + f

# Filter out the outliers and inspect them!
sample_dset = sample_dset[(sample_dset >= lower_fence) & (sample_dset <= upper_fence)]
print(sample_dset)

0       106.145874
1        91.044219
2       107.202858
3        86.464344
4       103.249059
5        90.923516
6       104.537804
7        98.168093
8       100.619681
9        88.562850
10       98.669220
11      101.294004
12       95.603203
13       96.755166
14      100.191893
15       98.559921
16       97.932122
17       90.858907
18       87.431719
19       98.345744
20      101.390562
21      106.771191
22      108.409304
23      106.708683
24       95.973145
26      108.147891
28       93.369981
29      103.094504
31      107.474869
32      104.207472
           ...    
980      94.054032
981      88.907025
982      97.331565
983     104.425545
984      88.668980
985     103.970487
986      87.387106
987      96.095900
988     102.087644
989     105.168744
990      98.189401
991      96.683125
992     106.880129
993      88.659658
995     104.805334
996      95.222647
997     100.399599
998      89.079794
1004    106.480217
1008     90.398271
1023    100.802027
1027     86.

Great! That works, but it isn't efficient to calculate this manually every time we run across a new data set.  

**TASK:** Write a function that takes in a pandas series, and returns a new pandas series with the outliers removed!

In [7]:
def remove_outliers(series):
    # Get Locations for Q1 and Q3
    q1 = series.quantile(.25)
    q3 = series.quantile(.75)
    f = (q3 - q1) * 1.5

    # calculate fence locations
    lower_fence = q1 - f
    upper_fence = q3 + f

    # Filter out the outliers and inspect them!
    return series[(series >= lower_fence) & (series <= upper_fence)]

<center><h2>Sample Size and Confidence Intervals</h2></center>

## What is a Confidence Interval?

Recall that in statistics, we almost never get to work with the entire population.  Instead, we work with samples taken from the population, and use statistical methods to try and estimates about the population based on what we see in the samples. When you think about this estimation process, this leads to two very important questions:

<center>1. **_How accurate are our estimates?_**</center>
<br>
<center>2. **_How many samples do we need to be need to be sure our estimates are accurate?_**</center>

This is where confidence intervals come in to play.  When estimating population parameters such as the population mean, for example, it is impossible to know with certainty that our estimate is 100% accurate.  Instead, statisticians define an acceptable margin of error.  In plain English, that means that we're okay with our estimate being wrong, as long as we're {X}% sure we're within a certain distance from the mean.  

To illustrate this concept, let's look at a type of graph statisticians use to denote confidence intervals, called a **_Box Plot_**.  

<center><img src='http://www.cs.utsa.edu/~cs1173/lessons/BoxPlotQuestions/BoxPlotQuestions_02.png'></center>

This is a box plot of the confidence intervals for the population means of 3 different types of Iris flowers (you'll get very familiar with this data set when you move onto supervised learning).  The only way that we could know the true mean of the sepal length of these three species of Irises is if we took the time to record the sepal length of every one of them *in the world*.  This isn't plausible.  Instead, we can use the data we've collected about our samples to determine upper and lower bounds for our confidence interval.  If we have an acceptable error rate (often refered to as an 'Alpha' value) of 5%, then that means that we have a confidence interval of 95%.  This means that we are 95% confident that the actual value of the population mean (often called the 'ground truth') is between our upper and lower bounds, which we find by using the confidence interval formula.  

<center><img src='img/Confidence_Interval_Formulas.png' height=60% width=60%></center>

Don't let the mathematical notation in those pictures scare you.  Here's what they each mean:

n = sample size
<br>
x_bar = mean of the sample
<br>
s = standard deviation of the sample
<br>
z* = point probability for that percentage (can be found with a lookup table or using the `scipy.stats` package)

**TASK:**  Read in the `iris.csv` data sets from the dataset folder. Make sure you specific that `header=None`, and se the `column names` variable to set the column names.  Compute the confidence intervals for at least one type of Iris flower.  

**STRETCH CHALLENGE #1:**  Write a function that takes in a Pandas Series and confidence level and returns the confidence interval.  (Hint: remember that each column in a dataframe is just a Series object!)

**STRETCH CHALLENGE #2:** Pick a column and visualize the the sample mean or median for at least one flower using a box-whisker plot.   (Hint: Consider writing the function from the first challenge to output everything needed for this visualization--then, this will be really easy!)

In [10]:
# Read in the dataset from iris.csv, in the datasets folder.  Make sure you pass 'header=None' and 'names=column_names'
# when calling pd.read_csv()!
column_names = ['Sepal Length(cm)', 'Sepal Width(cm)', 'Petal Length(cm)', 'Petal Width(cm)', 'Class']
df = pd.read_csv('./datasets/iris.csv', header=None, names=column_names)

In [18]:
import scipy.stats as st
print(df)

sample_size = len(df)
# sample_mean = None
# for i in range(0, sample_size):
#     sample_mean += 
# sample_std_dev = None
# z_star = st.norm.interval(0.95)
# ucl = None
# lcl = None

     Sepal Length(cm)  Sepal Width(cm)  Petal Length(cm)  Petal Width(cm)  \
0                 5.1              3.5               1.4              0.2   
1                 4.9              3.0               1.4              0.2   
2                 4.7              3.2               1.3              0.2   
3                 4.6              3.1               1.5              0.2   
4                 5.0              3.6               1.4              0.2   
5                 5.4              3.9               1.7              0.4   
6                 4.6              3.4               1.4              0.3   
7                 5.0              3.4               1.5              0.2   
8                 4.4              2.9               1.4              0.2   
9                 4.9              3.1               1.5              0.1   
10                5.4              3.7               1.5              0.2   
11                4.8              3.4               1.6              0.2   